In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import warnings
from pandas.plotting import scatter_matrix
from sklearn import metrics
import seaborn as sn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RepeatedStratifiedKFold
from skopt import BayesSearchCV
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, SMOTENC, SVMSMOTE, KMeansSMOTE, ADASYN
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline
from data_processing import (identify_merger, clean_hcris_before_2010, imputation_method, resampling)

In [2]:
# ignore warnings
warnings.filterwarnings("ignore")
# Set the display options
pd.set_option('display.max_columns',200)
pd.set_option('display.max_rows',200)
pd.options.mode.use_inf_as_na = True

# Data preparation

Firstly, I upload the verified data from Lina and identify the potential mergers (details see identify_merger() in data_processing.py).\
Return the data after identification as a dataframe (df1).\
Drop the 'Year of change' after 2012.

In [3]:
verified_data = identify_merger()
verified_data.drop(verified_data[verified_data['Year of change']==2017].index,inplace=True)
verified_data.drop(verified_data[verified_data['Year of change']==2016].index,inplace=True)
verified_data.drop(verified_data[verified_data['Year of change']==2015].index,inplace=True)
verified_data.drop(verified_data[verified_data['Year of change']==2014].index,inplace=True)
verified_data.drop(verified_data[verified_data['Year of change']==2013].index,inplace=True)

         id  Type of closure  \
0     10009              1.0   
1     10010              5.0   
2     10015              1.0   
3     10025              4.0   
4     10027              1.0   
..      ...              ...   
365  670018              1.0   
366  670021              4.0   
367  670029              5.0   
368  670052              4.0   
369  670066              4.0   

    Type of change (Convert/close/merge/change names)  Year of change  merged  
0                                               Close          2012.0       0  
1                                                Open             NaN       0  
2                                               Close          2011.0       0  
3                                         Open/merger          2013.0       1  
4                                               Close          2013.0       0  
..                                                ...             ...     ...  
365      Close, reopen in 2015 but probably partially  

Upload HCRIS data 2001-2012

In [4]:
# HCRIS 2001-2012 except 2010
hcris_2012 = pd.read_csv('ime_gme2552_10_2012.csv')
hcris_2011 = pd.read_csv('ime_gme2552_10_2011.csv')
hcris_2009 = pd.read_csv('ime_gme2552_96_2009.csv')
hcris_2008 = pd.read_csv('ime_gme2552_96_2008.csv')
hcris_2007 = pd.read_csv('ime_gme2552_96_2007.csv')
hcris_2006 = pd.read_csv('ime_gme2552_96_2006.csv')
hcris_2005 = pd.read_csv('ime_gme2552_96_2005.csv')
hcris_2004 = pd.read_csv('ime_gme2552_96_2004.csv')
hcris_2003 = pd.read_csv('ime_gme2552_96_2003.csv')
hcris_2002 = pd.read_csv('ime_gme2552_96_2002.csv')
hcris_2001 = pd.read_csv('ime_gme2552_96_2001.csv')
# HCRIS 2010
hcris_2010_1 = pd.read_csv('ime_gme2552_10_2010.csv')
hcris_2010_2 = pd.read_csv('ime_gme2552_96_2010.csv')
hcris_2010 = pd.merge(hcris_2010_1, hcris_2010_2, how='outer')

HCRIS_2012 data as another dataframe (df2).

In [5]:
# Match the column names with df1
hcris_2012.rename(columns={'provider': 'id'}, inplace=True)
print(f"The shape of original HCRIS 2012:{hcris_2012.shape}")

The shape of original HCRIS 2012:(6227, 41)


Check how many common (merged) hospitals between verified and HCRIS.

In [6]:
verified_hcris_common = pd.merge(verified_data, hcris_2012)
verified_hcris_common_merged = verified_hcris_common.loc[verified_hcris_common['merged'].isin([1])]
print(f'Common hospitals: {verified_hcris_common.shape}')
print(f'Common merged hospitals: {verified_hcris_common_merged.shape}')
verified_hcris_common_merged.head(20)

Common hospitals: (122, 45)
Common merged hospitals: (56, 45)


,id,Type of closure,Type of change (Convert/close/merge/change names),Year of change,merged,prvdr_num,fyb,fybstr,fye,fyestr,status,hospital_name,street_addr,po_box,city,state,zip_code,county,ime1,ime2,ime3,dsh1,dsh2,dsh3,dshpct,gme_part_a,gme_part_b,beds,bed_days,medicare_days,medicaid_days,days,intres,employees,volunteers,medicare_disch,medicaid_disch,discharges,medicaid_hmo_discharges,medicaid_hmo_ipf_subprovider,medicaid_hmo_irf_subprovider,medicaid_ipf_subprovider,medicaid_irf_subprovider,total_discharges_subprovider_ipf,total_discharges_subprovider_irf
2,10137,3.0,"Service reduction, specialty clinics",NaN,1,10137,10/1/2011,01-OCT-11,9/30/2012,30-SEP-12,As Submitted,COOPER GREEN MERCY HOSPITAL,1515 SIXTH AVENUE SOUTH,NaN,BIRMINGHAM,AL,35233,JEFFERSON,55245.0,NaN,NaN,384237.0,NaN,NaN,0.4494,111732.0,74964.0,149.0,54385.0,792.0,6456.0,12741.0,25.000000,650.150024,NaN,153.0,1783.0,2488.0,NaN,NaN,NaN,NaN,NaN,62.0,NaN
6,30100,3.0,"Open, merger",NaN,1,30100,10/1/2011,01-OCT-11,6/30/2012,30-JUN-12,Reopened,CARONDELET HEART AND VASCULAR INSTIT,4888 NORTH STONE DRIVE,NaN,TUCSON,AZ,85704-,PIMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,16440.0,2692.0,513.0,6563.0,NaN,219.309998,NaN,615.0,155.0,1637.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,30100,3.0,"Open, merger",NaN,1,30100,7/1/2012,01-JUL-12,11/7/2012,07-NOV-12,Settled,CARONDELET HEART AND VASCULAR INSTIT,4888 NORTH STONE DRIVE,NaN,TUCSON,AZ,85704-,PIMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,7800.0,584.0,143.0,1487.0,NaN,175.119995,NaN,135.0,42.0,414.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,50016,3.0,Open/merged multiple times,NaN,1,50016,7/1/2012,01-JUL-12,1/9/2013,09-JAN-13,Settled,ARROYO GRANDE COMMUNITY HOSPITAL,345 SOUTH HALCYON ROAD,NaN,ARROYO GRANDE,CA,93420,SAN LUIS OBISPO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.0,19345.0,2067.0,99.0,7578.0,NaN,255.000000,NaN,573.0,19.0,2273.0,NaN,NaN,NaN,NaN,4.0,NaN,369.0
17,50752,4.0,"Open, name change",NaN,1,50752,1/1/2012,01-JAN-12,12/31/2012,31-DEC-12,Settled,BROTMAN MEDICAL CENTER,3828 DELMAS TERRACE,NaN,CULVER CITY,CA,90232,LOS ANGELES,NaN,NaN,NaN,7086722.0,NaN,NaN,0.2970,NaN,NaN,283.0,103578.0,16896.0,5263.0,35870.0,NaN,607.000000,NaN,2893.0,894.0,7021.0,NaN,NaN,NaN,756.0,11.0,1736.0,417.0
18,70001,4.0,"Open, merger and name change",NaN,1,70001,10/1/2011,01-OCT-11,9/11/2012,11-SEP-12,Settled,HOSPITAL OF SAINT RAPHAEL,1450 CHAPEL STREET,NaN,NEW HAVEN,CT,06511,NEW HAVEN,15952772.0,NaN,NaN,6427300.0,NaN,NaN,0.0853,6012769.0,1411530.0,411.0,142617.0,43996.0,10199.0,90680.0,127.730003,2954.719971,NaN,7743.0,2886.0,18852.0,NaN,NaN,NaN,246.0,NaN,712.0,379.0
19,70015,4.0,"Open, merger and name change",NaN,1,70015,10/1/2011,01-OCT-11,9/30/2012,30-SEP-12,Settled,NEW MILFORD HOSPITAL,21 ELM STREET,NaN,NEW MILFORD,CT,06776,UNITED STATES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.0,31110.0,4591.0,822.0,8537.0,NaN,429.799988,5.26,1008.0,244.0,2288.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,110219,4.0,"Open, name change",NaN,1,110219,6/1/2012,01-JUN-12,12/31/2012,31-DEC-12,Reopened,SOUTH FULTON MEDICAL CENTER,1170 CLEVELAND AVE,NaN,EAST POINT,GA,30344,FULTON,62535.0,NaN,NaN,1923494.0,NaN,NaN,0.3551,116885.0,32265.0,198.0,42372.0,4546.0,4697.0,14984.0,6.130000,453.799988,NaN,789.0,888.0,2831.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,140051,4.0,"Open, name change",NaN,1,140051,10/1/2011,01-OCT-11,9/30/2012,30-SEP-12,Settled,SKOKIE HOSPITAL,9600 GROSS POINT ROAD,NaN,SKOKIE,IL,60076,COOK,2666080.0,NaN,NaN,2955137.0,NaN,NaN,0.0915,1098461.0,429646.0,128.0,52096.0,19373.0,4579.0,31015.0,20.379999,762.799988,NaN,3811.0,409.0,6644.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,150003,4.0,"Open, name change",NaN,1,150003,1/1/2012,01-JAN-12,12/31/2012,31-DEC-12,Reopened,ST. ELIZABETH CENTRAL,1501 HARTFORD STREET,NaN,LAFAYETTE,IN,47904-,TIPPECANOE,NaN,NaN,NaN,305820.0,101619.0,NaN,0.0442,NaN,NaN,94.0,34404.0,6966.0,1521.0,11913.0,NaN,535.570007,NaN,1609.0,406.0,2910.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Merge two dataframes by hospital ids.

In [7]:
verified_hcris_data = pd.merge(verified_data, hcris_2012, how='outer')
# Fill the value of column 'merged' in hosp_data not included in verified_data with 0
verified_hcris_data = verified_hcris_data.fillna({'merged':0})
print(verified_hcris_data.shape)

(6408, 45)


# Data cleaning

Drop duplicates and keep the latest date information.

In [8]:
# Return DataFrame with duplicate rows removed except for the id with the recent date.
verified_hcris_data.drop_duplicates(subset=['id'], keep='last', inplace=True)
verified_hcris_data.sort_values(by='id', ascending=True, inplace = True)
verified_hcris_data.reset_index(drop=True, inplace = True)
print(verified_hcris_data.shape)
verified_hcris_data.head(20)

(6313, 45)


,id,Type of closure,Type of change (Convert/close/merge/change names),Year of change,merged,prvdr_num,fyb,fybstr,fye,fyestr,status,hospital_name,street_addr,po_box,city,state,zip_code,county,ime1,ime2,ime3,dsh1,dsh2,dsh3,dshpct,gme_part_a,gme_part_b,beds,bed_days,medicare_days,medicaid_days,days,intres,employees,volunteers,medicare_disch,medicaid_disch,discharges,medicaid_hmo_discharges,medicaid_hmo_ipf_subprovider,medicaid_hmo_irf_subprovider,medicaid_ipf_subprovider,medicaid_irf_subprovider,total_discharges_subprovider_ipf,total_discharges_subprovider_irf
0,10001,NaN,NaN,NaN,0.0,10001.0,10/1/2011,01-OCT-11,9/30/2012,30-SEP-12,Amended,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,6987,DOTHAN,AL,36301,HOUSTON,NaN,NaN,NaN,8633626.0,NaN,NaN,0.1498,NaN,NaN,408.0,149328.0,42785.0,15986.0,87760.0,NaN,2160.600098,NaN,7790.0,3931.0,17435.0,NaN,NaN,NaN,NaN,NaN,215.0,NaN
1,10005,NaN,NaN,NaN,0.0,10005.0,10/1/2011,01-OCT-11,9/30/2012,30-SEP-12,Reopened,MARSHALL MEDICAL CENTER - SOUTH,2505 U.S. HIGHWAY 431,NaN,BOAZ,AL,35957-,MARSHALL,NaN,NaN,NaN,2891397.0,NaN,NaN,0.1253,NaN,NaN,204.0,74664.0,17787.0,6381.0,36415.0,NaN,1056.020020,NaN,3797.0,1291.0,9168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10006,NaN,NaN,NaN,0.0,10006.0,7/1/2012,01-JUL-12,6/30/2013,30-JUN-13,Reopened,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,818,FLORENCE,AL,35630,LAUDERDALE,NaN,NaN,NaN,3498027.0,NaN,NaN,0.0951,NaN,NaN,358.0,130670.0,28599.0,7909.0,52665.0,NaN,968.530029,NaN,5039.0,1765.0,10862.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10007,NaN,NaN,NaN,0.0,10007.0,10/1/2011,01-OCT-11,9/30/2012,30-SEP-12,Settled,MIZELL MEMORIAL HOSPITAL,702 MAIN STREET,429,OPP,AL,36462-,COVINGTON,NaN,NaN,NaN,185865.0,NaN,NaN,0.0533,NaN,NaN,78.0,28548.0,4122.0,619.0,5867.0,NaN,153.449997,NaN,819.0,133.0,1302.0,NaN,NaN,NaN,NaN,NaN,262.0,NaN
4,10008,NaN,NaN,NaN,0.0,10008.0,1/1/2012,01-JAN-12,12/31/2012,31-DEC-12,Amended,CRENSHAW COMMUNITY HOSPITAL,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,CRENSHAW,NaN,NaN,NaN,287870.0,NaN,NaN,0.1713,NaN,NaN,29.0,10614.0,1764.0,668.0,3240.0,NaN,106.699997,NaN,374.0,166.0,744.0,NaN,NaN,NaN,104.0,NaN,540.0,NaN
5,10009,1.0,Close,2012.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10010,5.0,Open,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,10011,NaN,NaN,NaN,0.0,10011.0,7/1/2012,01-JUL-12,6/30/2013,30-JUN-13,Amended,ST. VINCENTS EAST,50 MEDICAL PARK DRIVE EAST,NaN,BIRMINGHAM,AL,35235-,JEFFERSON,1670142.0,NaN,NaN,2023583.0,NaN,NaN,0.0612,787021.0,229995.0,279.0,101835.0,23284.0,10013.0,75795.0,14.660000,1101.239990,NaN,4117.0,1683.0,14380.0,NaN,NaN,NaN,27.0,4.0,421.0,232.0
8,10012,NaN,NaN,NaN,0.0,10012.0,4/1/2012,01-APR-12,3/31/2013,31-MAR-13,Settled,DEKALB REGIONAL MEDICAL CENTER,200 MEDICAL CENTER DRIVE,NaN,FORT PAYNE,AL,35967,DEKALB,NaN,NaN,NaN,928833.0,NaN,NaN,0.1200,NaN,NaN,97.0,35405.0,5771.0,2818.0,13469.0,NaN,371.500000,NaN,1416.0,844.0,3758.0,NaN,NaN,NaN,11.0,NaN,332.0,NaN
9,10015,1.0,Close,2011.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


After combining the two dataframes, I found some hospitals with blank information for almost all columns except 'id' and 'merged'.\
They're likely to close or merge before year 2012.\
So fill those hospitals information with previous HCRIS data.

Clean the previous HCRIS data.

In [ ]:
# 2011
hcris_2011.rename(columns={'provider': 'id'}, inplace=True)
hcris_2011.drop_duplicates(subset=['id'], keep='last', inplace=True)
hcris_2011.sort_values(by='id', ascending=True, inplace=True)
hcris_2011.reset_index(drop=True, inplace=True)
# 2010
hcris_2010.rename(columns={'provider': 'id'}, inplace=True)
hcris_2010[hcris_2010["id"].duplicated(keep="last") == True]
hcris_2010.drop_duplicates(subset=['id'], keep='last', inplace=True)
hcris_2010.sort_values(by='id', ascending=True, inplace=True)
hcris_2010.reset_index(drop=True, inplace=True)
# 
hcris_2009 = clean_hcris_before_2010(hcris_2009)
hcris_2008 = clean_hcris_before_2010(hcris_2008)
hcris_2007 = clean_hcris_before_2010(hcris_2007)
hcris_2006 = clean_hcris_before_2010(hcris_2006)
hcris_2005 = clean_hcris_before_2010(hcris_2005)
hcris_2004 = clean_hcris_before_2010(hcris_2004)
hcris_2003 = clean_hcris_before_2010(hcris_2003)
hcris_2002 = clean_hcris_before_2010(hcris_2002)
hcris_2001 = clean_hcris_before_2010(hcris_2001)